In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv('../data/dummy_train.csv')
test = pd.read_csv('../data/dummy_test.csv')

In [3]:
train.head()

,row_id,date,num_sold,Finland,Norway,Sweden,KaggleMart,KaggleRama,Kaggle Hat,Kaggle Mug,Kaggle Sticker
0,0,2015-01-01,329,1,0,0,1,0,0,1,0
1,1,2015-01-01,520,1,0,0,1,0,1,0,0
2,2,2015-01-01,146,1,0,0,1,0,0,0,1
3,3,2015-01-01,572,1,0,0,0,1,0,1,0
4,4,2015-01-01,911,1,0,0,0,1,1,0,0


In [4]:
train.dtypes

row_id             int64
date              object
num_sold           int64
Finland            int64
Norway             int64
Sweden             int64
KaggleMart         int64
KaggleRama         int64
Kaggle Hat         int64
Kaggle Mug         int64
Kaggle Sticker     int64
dtype: object

In [5]:
#creating the train and validation set
train_df = train[:int(0.8*(len(train)))].drop(columns=['row_id'])
valid = train[int(0.8*(len(train))):].drop(columns=['row_id'])

In [6]:
train_df.dtypes

date              object
num_sold           int64
Finland            int64
Norway             int64
Sweden             int64
KaggleMart         int64
KaggleRama         int64
Kaggle Hat         int64
Kaggle Mug         int64
Kaggle Sticker     int64
dtype: object

In [7]:
train_df['date'] = pd.to_datetime(train_df['date'])

In [8]:
train_df.dtypes

date              datetime64[ns]
num_sold                   int64
Finland                    int64
Norway                     int64
Sweden                     int64
KaggleMart                 int64
KaggleRama                 int64
Kaggle Hat                 int64
Kaggle Mug                 int64
Kaggle Sticker             int64
dtype: object

In [9]:
np.asarray(train_df)

array([[Timestamp('2015-01-01 00:00:00'), 329, 1, ..., 0, 1, 0],
       [Timestamp('2015-01-01 00:00:00'), 520, 1, ..., 1, 0, 0],
       [Timestamp('2015-01-01 00:00:00'), 146, 1, ..., 0, 0, 1],
       ...,
       [Timestamp('2018-03-14 00:00:00'), 321, 0, ..., 0, 0, 1],
       [Timestamp('2018-03-14 00:00:00'), 247, 0, ..., 0, 1, 0],
       [Timestamp('2018-03-14 00:00:00'), 491, 0, ..., 1, 0, 0]],
      dtype=object)

In [18]:
train_df.set_index('date', inplace= True)

In [22]:
train_df.index = pd.DatetimeIndex(train_df.index).to_period('D')

In [19]:
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
# Call the function and run the test

adf_test(train_df)

Results of Dickey-Fuller Test:


ValueError: x is required to have ndim 1 but has ndim 2

In [25]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train_df.drop(columns = ['num_sold']), exog = train_df['num_sold'])
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/Users/bramblepatch/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


ValueError: Please provide an exog_future argument to the forecast method.

In [ ]:
#converting predictions to dataframe
cols = train.drop(columns = 'row_id').columns
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(pred[i], valid[i])))